In [67]:
import pandas as pd
import re
import sys

In [69]:
#Modulo que  agrega el path de los modulos al sys.path
from pathlib import Path

# Obtener la ruta al directorio "modulos"
ruta_modulos = Path("../modulos").resolve()

# Agregar la ruta al sys.path
if str(ruta_modulos) not in sys.path:
    sys.path.append(str(ruta_modulos))          

# Obtener la ruta al directorio "modulos"
ruta_modulos = Path("../modulos").resolve()
print(f"Ruta de modulos: {ruta_modulos}")  # Verifica la ruta que se obtiene


Ruta de modulos: C:\Users\OPERADOR\Desktop\PROYECTO DIGITALIZACION\digitalizacion_distriapp\backend\src\modulos


In [70]:
# Carga de datos
from utilidades.pandas_funciones import cargar_datos
df_proveedor = cargar_datos("prueba2.csv")
df_productos = cargar_datos("p_STProPrv.csv")

In [50]:
#Copias del DataFrame 
copy_dfProductos = df_productos
copy_dfProveedores = df_proveedor

In [53]:
copy_dfProductos["des_prod"]

0                                FADA MESNA 200 mg a.x 10
1                                    BOTOX 100 U vial x 1
2                                    BOTOX 200 U vial x 1
3                             ENSURE vainilla pvo.x 400 g
4                     ENSURE ADVANCE vainilla pvo.x 400 g
                              ...                        
5618           MASCARA BUCO-NASAL YF02.T:S  - MARCA YUWEL
5619                            LIDOXA jalea x 20 x 25 ml
5620    AGALSIDASA ALFA 1MG/ML F.A. X3.5ML - REPLAGAL ...
5621                            JEVITY PLUS bot.x 1000 ml
5622                CAL SODADA DRAGER DRAGERSORB 800 PLUS
Name: des_prod, Length: 5623, dtype: object

In [44]:
copy_dfProveedores["DESCRIPCIÓN"]

0     HUMIRA AC x 2 (20mg)\nHUMIRA AC x 2 (40mg) HUM...
1                  RINVOQ 15MG RINVOQ 30MG\nRINVOQ 45MG
2                              SKYRIZI SKYRIZI\nSKYRIZI
3     KALETRA SOLUCIÓN\nKALETRA COMPRIMIDOS KALETRA ...
4                                             RITONAVIR
5                                               MAVIRET
6              VENCLEXTA VENCLEXTA VENCLEXTA\nVENCLEXTA
7                         SEVORANE\nSEVORANE QUICK FILL
8                                    SURVANTA\nSURVANTA
9                           ZEMPLAR VIAL\nZEMPLAR 2 MCG
10                                         BOTOX\nBOTOX
11                                              OZURDEX
Name: DESCRIPCIÓN, dtype: object

In [61]:
import re

def clear_column(column):
    """
    Limpiar la columna de descripciones para eliminar términos irrelevantes y normalizar el texto.
    """
    # Expresión regular optimizada para eliminar números, unidades, abreviaciones, y términos irrelevantes
    columna_limpia = re.sub(r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b', '', column, flags=re.IGNORECASE)
    
    # Eliminar caracteres especiales como los dos puntos, puntos finales, guiones, comas, etc.
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)

    # Eliminar múltiples espacios en blanco y normalizar la columna
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()

    # Convertir todo el texto a minúsculas
    columna_limpia = columna_limpia.lower()

    # Eliminar posibles palabras vacías adicionales que puedan quedar
    columna_limpia = re.sub(r'\b(?:de|la|el|en|a|y|con|para|del|sobre|en|por|para|con|medicamento|tratamiento|ac)\b', '', columna_limpia).strip()

    # Opción adicional para quitar palabras repetidas o redundantes (como 'skyrizi skyrizi')
    columna_limpia = re.sub(r'\b(\w+)\s+\1\b', r'\1', columna_limpia)

    # Eliminar términos relacionados con la variabilidad en dosis o presentación (por ejemplo, "20mg", "30mg", etc.)
    columna_limpia = re.sub(r'\b(\d+mg|\d+g|\d+ml)\b', '', columna_limpia)

    return columna_limpia


# Función para obtener coincidencias entre las columnas y agregar una columna 'match'
def obtener_coincidencias_con_match(productos_df, proveedores_df, col_productos, col_proveedores):
    """
    Realiza un match entre las columnas especificadas de dos DataFrames y agrega una columna 'match'.
    
    Parámetros:
        productos_df (DataFrame): DataFrame de productos.
        proveedores_df (DataFrame): DataFrame de proveedores.
        col_productos (str): Nombre de la columna en productos.
        col_proveedores (str): Nombre de la columna en proveedores.
        
    Retorna:
        DataFrame: DataFrame con las coincidencias y una columna 'match'.
    """
    # Limpiar ambas columnas antes de realizar la comparación
    productos_df[col_productos] = productos_df[col_productos].apply(lambda x: clear_column(str(x)))
    proveedores_df[col_proveedores] = proveedores_df[col_proveedores].apply(lambda x: clear_column(str(x)))
    
    resultados = []

    # Iterar por cada descripción en proveedores
    for _, proveedor in proveedores_df.iterrows():
        descripcion_proveedor = proveedor[col_proveedores]
        
        # Buscar coincidencias en los productos
        coincidencias = productos_df[
            productos_df[col_productos].str.contains(descripcion_proveedor, case=False, na=False)
        ]
        
        # Agregar coincidencias al resultado
        if not coincidencias.empty:
            for _, producto in coincidencias.iterrows():
                resultado = {**proveedor.to_dict(), **producto.to_dict(), "match": True}
                resultados.append(resultado)
        else:
            # Si no hay coincidencias, agregar el proveedor con valores nulos para los productos
            resultado = proveedor.to_dict()
            for col in productos_df.columns:
                resultado[col] = None
            resultado["match"] = False
            resultados.append(resultado)
    
    return pd.DataFrame(resultados)

# Llamar Match a la función con tus DataFrames
resultado_df = obtener_coincidencias_con_match(
    productos_df=copy_dfProductos,
    proveedores_df=copy_dfProveedores,
    col_productos="des_prod",
    col_proveedores="DESCRIPCIÓN"
)


In [63]:
# Mostrar el DataFrame resultante
resultado_df

,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,niprod,cod_prod,des_prod,...,cod_barrpr,fecvig,cost_usual,cost_ext,plazo_pago,pc_pv,prec_uc,prec_uc_ext,preferido,match
0,20015888\n20005289\n20021980,humira humira,"2 JERINGAS PRELL. X 0,2 ml\n2 LAPICERAS PRELL....",8054083017068\n8054083014050\n8054083017648,"$ 740,145.59\n$ 1,480,292....","$ 611,690.58\n$ 1,223,382...","$ 1,073,211.02\n$ 2,146,42...",NaN,None,None,...,NaN,None,None,None,NaN,None,None,None,NaN,False
1,20062091\n20069005\n20078725,rinvoq rinvoq,15 MG x 30 COMP REC DE LIB. PROLONGADA\n30 MG ...,8054083018706\n8054083021386\n8054083023922,"$ 1,799,606.71\n$ 3,184,521.02...","$ 1,487,278.28\n$ 2,631,835....","$ 2,609,429.58\n$ 4,617,55...",NaN,None,None,...,NaN,None,None,None,NaN,None,None,None,NaN,False
2,20069017\n20079636\n20079635,skyrizi,1 LAPICERA PRELLENADA por 1 ML (150 mg)\n360mg...,8054083021935\n8054083023311\n8054083023328,"$ 4,344,159.74\n$ 4,561,367.72...","$ 3,590,214.67\n$ 3,769,725....","$ 6,299,031.64\n$ 6,613,98...",NaN,None,None,...,NaN,None,None,None,NaN,None,None,None,NaN,False
3,20058578\n20025520\n20012232,kaletra solución kaletra comprimidos kaletra c...,160 ML\n60 COMP. RECUBIERTOS\n120 COMP. RECUBI...,8054083006406\n8054083003382\n8054083003474,"$ 148,177.93\n$ 138,771...","$ 122,461.09\n$ 114,686...","$ 214,857.98\n$ 201,...",NaN,None,None,...,NaN,None,None,None,NaN,None,None,None,NaN,False
4,20011289,ritonavir,30 COMPRIMIDOS,8054083005003,"$ 25.166,11","$ 20.798,42","$ 36.490,79",119560.0,47909,ritonavir rec,...,NaN,24/11/2023,0,0,0.0,0,"9365,206","9365,206",0.0,True
5,20011756,maviret,100 MG/40 MG x 84 COMP REC,8054083015330,"$ 9.150.200,87","$ 7.562.149,48","$ 13.267.791,31",119558.0,55552,maviret rec,...,NaN,20/09/2022,0,0,0.0,0,617100,617100,0.0,True
6,20019068\n20019067\n20060375\n20060376,venclexta,KIT DE INICIO X 1 - PACK 4X7 DIAS 100 MG X 120...,8054083013343\n8054083013336\n8054083013329\n8...,"$ 2,420,724.48\n$ 11,215,990.12\...","$ 2,000,598.69\n$ 9,269,413....","$ 3,510,050.42\n$ 16,263,185....",119573.0,53120,venclexta,...,NaN,24/10/2024,0,0,0.0,0,"12940550,06","12940550,06",0.0,True
7,20019068\n20019067\n20060375\n20060376,venclexta,KIT DE INICIO X 1 - PACK 4X7 DIAS 100 MG X 120...,8054083013343\n8054083013336\n8054083013329\n8...,"$ 2,420,724.48\n$ 11,215,990.12\...","$ 2,000,598.69\n$ 9,269,413....","$ 3,510,050.42\n$ 16,263,185....",119575.0,53119,venclexta kit inicio,...,NaN,16/09/2022,0,0,0.0,0,"261047,28","261047,28",0.0,True
8,20012619\n20012621,sevorane quick fill,250 ML\n250 ML,8054083015156\n8054083015163,"$ 158,394.74\n$ 158,39...","$ 130,904.73\n$ 130,90...","$ 229,672.36\n$ 229,...",119563.0,50933,sevorane quick fill,...,NaN,17/10/2024,0,0,0.0,0,"165786,0325","165786,0325",0.0,True
9,20012619\n20012621,sevorane quick fill,250 ML\n250 ML,8054083015156\n8054083015163,"$ 158,394.74\n$ 158,39...","$ 130,904.73\n$ 130,90...","$ 229,672.36\n$ 229,...",119563.0,50933,sevorane quick fill,...,NaN,18/07/2022,0,0,0.0,0,"12812,9","12812,9",0.0,True
